# Emaude Altema P24 SIM IFI

In [1]:
#Importation des libraries necessaires pour le travail.
import cv2 as cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt
import os
from scipy.spatial import distance
from IPython.display import Markdown, display
import pandas as pd

In [2]:
from sklearn.neighbors import KDTree
from sklearn.cluster import KMeans

In [13]:
from binarytree import tree, Node
from math import floor

from lshashpy3 import LSHash

In [4]:
def calcul_descripteurs(image_path, image_name):
    image = cv2.imread(os.path.join(image_path, image_name)) #pour lire chaque image de la base

    # division des couleurs de l'image en l'espace (BGR) 
    BGR_layers = cv2.split(image)

    # creation de l'Histogramme de chaque du dataset
    B_hist = cv2.calcHist(BGR_layers, [0], None, [32], (0, 256), accumulate=False)
    G_hist = cv2.calcHist(BGR_layers, [1], None, [32], (0, 256), accumulate=False)
    R_hist = cv2.calcHist(BGR_layers, [2], None, [32], (0, 256), accumulate=False)
    histogramme = np.concatenate((B_hist, G_hist, R_hist), axis=None)

    # Moments de Hu pour decrire la forme de chaque image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hu_moment = cv2.HuMoments(cv2.moments(image))
    hu_moment = hu_moment.flatten()
    
    return histogramme, hu_moment
route = ('C:/Users/Administrator/TP2_SIFT/coil-100')

In [5]:
# Lecture des images stockés sur le disque dur, calculer leurs histogrammes et leurs moment de Hu
# ici je cree 3 tableau pour recevoir 1 pour recevoir les histogrammes, 2 pour recevoir la liste des moments de HU
# et le 3eme pour contenir la liste des labels des images, 

liste_histogrammes = []
liste_hu_moment = []
label_images = []

for img in [f for f in os.listdir('C:/Users/Administrator/TP2_SIFT/coil-100')]:#la boucle for pour parcourir toute la base d'image
    if img.split('.')[1] == 'png': # pour verifier que c'est une image
        histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', img)#j'appelle la fonction qui 
        # calcul de l'histogramme et le moment de hu de chaque image de  la base
        liste_histogrammes.append(histogramme) # on sauvegarde chque hystogramme dans le tableau liste_histogrammes
        liste_hu_moment.append(hu_moment)# on sauvegarde chaque moment de hu dans le table liste_hu_moment
        label_images.append(img)  # Pour recuperer le nom de  chaque image de la base
       
        

In [6]:
# Sauvegarde des caractéristiques telles que les histogrammes, les moment de HU et les noms de toutes les images
np.save('liste_histogrammes.npy', np.array(liste_histogrammes))
np.save('liste_hu_moment.npy', np.array(liste_hu_moment))
np.save('label_images.npy', np.array(label_images))


# Recherche des images les plus proches a partir d'une image

In [7]:
# appel des caractéristiques qu'on a cree plus haut
liste_histogrammes = np.load('liste_histogrammes.npy')
liste_hu_moment = np.load('liste_hu_moment.npy')
label_images = np.load('label_images.npy')

In [8]:
w1 = 0.7
w2 = 0.3

In [9]:
# On choisit une image n'importe laquelle ("je prends obj1__5 ")
image_entree = "obj1__5.png"

#j'appelle la fonction qui calcul de l'histogramme et le moment de hu pour l'image d'entree 
histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', image_entree)
# J'implemente la fonction KDTre de SKlearn.neighbors
tree = KDTree(liste_histogrammes, leaf_size=2)
dist, ind = tree.query([histogramme], k=100)

# Tableau contenant les distances
liste_distances = []

# Calcul des distances entre l'image d'entree et les images de toute la base
for i in ind[0]: # on parcourt chaque image de la base
    distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
    # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
    distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
    #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
    distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
    dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
    liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances
        
# Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
liste_distances = sorted(liste_distances, key=lambda x: x[1])

print("Exemple : {}".format(image_entree))
print()
# Ici je mets les 10 images les plus proches de l'image d'entree
print("Résultats de la recherche (N=10) :")
print("Image \t \t \t Distance".format(image_entree))
# ici je reccupere les 10 premieres images que j'ai trie et mis dans la liste_distance
for i in liste_distances[0:10]:
    # et la j'affiche chque image et sa distance
      print(" {} \t \t {}".format(label_images[i[0]], i[1]))

Exemple : obj1__5.png

Résultats de la recherche (N=10) :
Image 	 	 	 Distance
 obj1__5.png 	 	 0.0
 obj1__10.png 	 	 422.67865314048026
 obj1__0.png 	 	 492.9648707912255
 obj1__15.png 	 	 924.9325045435414
 obj1__355.png 	 	 1190.6415524935658
 obj1__20.png 	 	 1777.4890180499847
 obj1__180.png 	 	 1928.8828990703612
 obj1__190.png 	 	 1989.9590471682534
 obj1__185.png 	 	 2047.6344290034644
 obj1__175.png 	 	 2225.5437102036117


In [10]:
!pip install binarytree

In [11]:
# On choisit une image n'importe laquelle ("je prends obj1__5 ")
image_entree = "obj1__5.png"

#j'appelle la fonction qui calcul de l'histogramme et le moment de hu pour l'image d'entree 
histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', image_entree)

#ici j'implemente la fonction KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(liste_histogrammes)
labels = kmeans.labels_

label_entree = kmeans.predict(np.array([histogramme]))


# Tableau contenant les distances
liste_distances = []

# Calcul des distances entre l'image d'entree et les images de toute la base
for i in range(len(labels)): # on parcourt chaque image de la base
    if label_entree[0] == labels[i]:
        distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
        # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
        distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
        #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
        distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
        dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
        liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances
        
# Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
liste_distances = sorted(liste_distances, key=lambda x: x[1])

print("Exemple : {}".format(image_entree))
print()
# Ici je mets les 10 images les plus proches de l'image d'entree
print("Résultats de la recherche (N=10) :")
print("Image \t \t \t Distance".format(image_entree))
# ici je reccupere les 10 premieres images que j'ai trie et mis dans la liste_distance
for i in liste_distances[0:10]:
    # et la j'affiche chque image et sa distance
      print(" {} \t \t {}".format(label_images[i[0]], i[1]))

Exemple : obj1__5.png

Résultats de la recherche (N=10) :
Image 	 	 	 Distance
 obj1__5.png 	 	 0.0
 obj1__10.png 	 	 422.67865314048026
 obj1__0.png 	 	 492.9648707912255
 obj1__15.png 	 	 924.9325045435414
 obj1__355.png 	 	 1190.6415524935658
 obj1__20.png 	 	 1777.4890180499847
 obj1__180.png 	 	 1928.8828990703612
 obj1__190.png 	 	 1989.9590471682534
 obj1__185.png 	 	 2047.6344290034644
 obj1__175.png 	 	 2225.5437102036117


In [12]:
# J'installe lshashpy3 pour pouvoir utiliser la methode lsh
pip install lshashpy3

In [16]:
# On choisit une image n'importe laquelle ("je prends obj1__5 ")
image_entree = "obj1__5.png"

#j'appelle la fonction qui calcul de l'histogramme et le moment de hu pour l'image d'entree 
histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', image_entree)

Implementation de la methode LSH
lsh = LSHash(6, 96)

# index vector
lsh.index([i for i in range(96)]) # On note que chaque Histogramme a 96 valeurs

for h in liste_histogrammes:
    lsh.index(h)

top_n = 100 #pour choisir 100 images de mon dataset
nn = lsh.query(histogramme, num_results=top_n, distance_func="euclidean")

# Tableau contenant les distances
liste_distances = []

# Calcul des distances entre l'image d'entree et les images de toute la base
data = [list(i) for i in liste_histogrammes]
for ((vec,extra_data),dist) in nn:
    i = data.index(list(vec))
    distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
    # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
    distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
    #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
    distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
    dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
    liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances
        
# Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
liste_distances = sorted(liste_distances, key=lambda x: x[1])

print("Exemple : {}".format(image_entree))
print()
# Ici je mets les 10 images les plus proches de l'image d'entree
print("Résultats de la recherche (N=10) :")
print("Image \t \t \t Distance".format(image_entree))
# ici je reccupere les 10 premieres images que j'ai trie et mis dans la liste_distance
for i in liste_distances[0:10]:
    # et la j'affiche chque image et sa distance
      print(" {} \t \t {}".format(label_images[i[0]], i[1]))

Exemple : obj1__5.png

Résultats de la recherche (N=10) :
Image 	 	 	 Distance
 obj1__5.png 	 	 0.0
 obj1__10.png 	 	 422.67865314048026
 obj1__0.png 	 	 492.9648707912255
 obj1__15.png 	 	 924.9325045435414
 obj1__355.png 	 	 1190.6415524935658
 obj1__180.png 	 	 1928.8828990703612
 obj1__190.png 	 	 1989.9590471682534
 obj1__185.png 	 	 2047.6344290034644
 obj1__175.png 	 	 2225.5437102036117
 obj1__195.png 	 	 3256.303069313789


In [18]:
#tp = true positive, fp = false positive
tp = 0
fp = 0
tree = KDTree(liste_histogrammes, leaf_size=2)

for img in [f for f in os.listdir('C:/Users/Administrator/TP2_SIFT/evaluation')]:#la boucle for pour parcourir toute la base d'image
    if img.split('.')[1] == 'png': # pour verifier que c'est une image        
        histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', img)#j'appelle la fonction qui 
        label_image_entree = img.split('__')[0]

        dist, ind = tree.query([histogramme], k=100)

        # Tableau contenant les distances
        liste_distances = []

        # Calcul des distances entre l'image d'entree et les images de toute la base
        for i in ind[0]: # on parcourt chaque image de la base
            distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
            # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
            distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
            #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
            distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
            dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
            liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances
        
            # Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
            liste_distances = sorted(liste_distances, key=lambda x: x[1])
           
        # je calcule les true et false positives
            for i in liste_distances[0:10]:
                if label_image_entree == label_images[i[0]].split('__')[0]:
                    tp += 1
                else:
                    fp += 1
         # Je calcule la precision pour la methode KDTree           
print('La precision est : {}'.format(tp/(tp+fp)))

La precision est : 0.9733367386029881


In [20]:
tp = 0
fp = 0
kmeans = KMeans(n_clusters=10, random_state=0).fit(liste_histogrammes)
labels = kmeans.labels_

for img in [f for f in os.listdir('C:/Users/Administrator/TP2_SIFT/evaluation')]:#la boucle for pour parcourir toute la base d'image
    if img.split('.')[1] == 'png': # pour verifier que c'est une image        
        histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', img)#j'appelle la fonction qui 
        label_image_entree = img.split('__')[0]

        label_entree = kmeans.predict(np.array([histogramme]))


        # Tableau contenant les distances
        liste_distances = []

        # Calcul des distances entre l'image d'entree et les images de toute la base
        for i in range(len(labels)): # on parcourt chaque image de la base
            if label_entree[0] == labels[i]:
                distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
                # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
                distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
                #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
                distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
                dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
                liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances

        # Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
        liste_distances = sorted(liste_distances, key=lambda x: x[1])
            
        for i in liste_distances[0:10]:
            if label_image_entree == label_images[i[0]].split('__')[0]:
                tp += 1
            else:
                fp += 1
           
         # Je calcule la precision pour la methode KMeans         
print('La precision est : {}'.format(tp/(tp+fp)))

La precision est : 0.9536585365853658


In [18]:
tp = 0
fp = 0
lsh = LSHash(6, 96)

# index vector
lsh.index([i for i in range(96)])

for h in liste_histogrammes:
    lsh.index(h)

top_n = 100

for img in [f for f in os.listdir('C:/Users/Administrator/TP2_SIFT/evaluation')]:#la boucle for pour parcourir toute la base d'image
    if img.split('.')[1] == 'png': # pour verifier que c'est une image        
        histogramme,hu_moment  =calcul_descripteurs('C:/Users/Administrator/TP2_SIFT/coil-100', img)#j'appelle la fonction qui 
        label_image_entree = img.split('__')[0]

        nn = lsh.query(histogramme, num_results=top_n, distance_func="euclidean")

        # Tableau contenant les distances
        liste_distances = []

        # Calcul des distances entre l'image d'entree et les images de toute la base
        data = [list(i) for i in liste_histogrammes]
        for ((vec,extra_data),dist) in nn:
            i = data.index(list(vec))
            distance_couleur = cv2.compareHist(histogramme, liste_histogrammes[i], 1)# calcul de la distance entre l'histogramme de l'image
            # et l'histogrammes de chaque image de la base en utilisant la methode chi_square
            distance_forme = distance.euclidean(hu_moment, liste_hu_moment[i])# calcul de la distance entre les moments de hu de l'image 
            #d'entree et les moments de hu de chaque image de la base en utilisant la distance euclidienne.
            distance_totale = distance_couleur * w1 + distance_forme * w2 # calcul de la distance totale en utilisant les poids W1 et W2
            dist = [i, distance_totale] #ici je sauvegarde l'indice de chaque l'image et la distance totale correspondante 
            liste_distances.append(dist)  # ic je mets mon tableau dist dans la liste des distances

        # Je trie le tableau par ordre croissant  c'est-a-dire je mets les plus petites valeurs en haut et les plus grandes en bas
        liste_distances = sorted(liste_distances, key=lambda x: x[1])
            
        for i in liste_distances[0:10]:
            if label_image_entree == label_images[i[0]].split('__')[0]:
                tp += 1
            else:
                fp += 1
                    
         # Je calcule la precision pour la methode LSH
print('La precision est : {}'.format(tp/(tp+fp)))

La precision est : 0.9240196078431373


# En conclusion La methode KDTree donne une plus grande precision ce qui confirme que cette methode est la meilleure si on fait une recherche sur cette base d'image.